# Gemini Document Verification Demo

This notebook demonstrates the core Gemini AI functionalities used in the Document Verification Assistant:

1. **Creating a Corpus** (File Search Store)
2. **Uploading Reference Documents** with AI-generated metadata
3. **Querying the Corpus** for information
4. **Verification Layer** - verifying document chunks against the corpus

## Prerequisites

```bash
pip install google-genai python-dotenv
```

## API Key Setup (choose one method)

**Option 1: .env file** (recommended)
```bash
echo "GEMINI_API_KEY=your_api_key_here" >> .env
```

**Option 2: Environment variable**
```bash
export GEMINI_API_KEY="your_api_key_here"
```

**Option 3: Set directly in notebook** (see next cell)

## Setup: Import Libraries and Initialize Client

In [ ]:
import os
import time
import json
from pathlib import Path
from dotenv import load_dotenv
from google import genai
from google.genai import types

# Load environment variables from .env file (if it exists)
load_dotenv()

# Get API key from .env, environment variable, or set manually below
api_key = os.getenv("GEMINI_API_KEY")

# OPTION 3: Set API key manually (uncomment and replace with your key)
# api_key = "your_api_key_here"

if not api_key:
    raise ValueError(
        "GEMINI_API_KEY not found. Please set it using one of these methods:\n"
        "  1. Create a .env file with: GEMINI_API_KEY=your_key\n"
        "  2. Set environment variable: export GEMINI_API_KEY=your_key\n"
        "  3. Uncomment the line above and set api_key directly"
    )

# Initialize Gemini client
client = genai.Client(api_key=api_key)
print("✓ Gemini client initialized successfully")

## Part 1: Creating a Corpus (File Search Store)

A **File Search Store** acts as a corpus - a searchable knowledge base of reference documents.

Key features:
- Automatically chunks and indexes documents
- Generates embeddings for semantic search
- Supports metadata filtering
- **Free** storage and query-time embedding (only pay for initial indexing)

In [ ]:
# Create a File Search store (corpus)
store_name_display = f"Demo Corpus - {int(time.time())}"

print(f"Creating File Search store: {store_name_display}")
store = client.file_search_stores.create(
    config={'display_name': store_name_display}
)

print(f"\n✓ Store created successfully!")
print(f"  Store ID: {store.name}")
print(f"  Display Name: {store.display_name}")

# Save store name for later use
CORPUS_STORE_NAME = store.name

## Part 2: Uploading Documents with AI Metadata

We'll:
1. Create a sample reference document
2. Use **Gemini Flash** to generate metadata (summary, keywords, document type)
3. Upload to the File Search store with metadata

In [ ]:
# Create a sample reference document
sample_doc_content = """SERVICE AGREEMENT

This Service Agreement ("Agreement") is entered into on January 15, 2024, between:

Company A ("Provider") - 123 Main Street, San Francisco, CA 94102
Company B ("Client") - 456 Market Street, San Francisco, CA 94103

TERMS AND CONDITIONS:

1. Services: Provider agrees to deliver software development services as specified in the Statement of Work.

2. Payment Terms: Client agrees to pay $150,000 for the services, payable in three installments:
   - $50,000 upon signing
   - $50,000 at project midpoint
   - $50,000 upon completion

3. Timeline: The project shall commence on February 1, 2024, and be completed by June 30, 2024.

4. Intellectual Property: All work product shall become the property of the Client upon final payment.

5. Confidentiality: Both parties agree to maintain confidentiality of proprietary information.

Signed:
John Smith, CEO, Company A
Jane Doe, CEO, Company B
"""

# Save to temporary file
import tempfile
temp_file = tempfile.NamedTemporaryFile(mode='w', suffix='.txt', delete=False, encoding='utf-8')
temp_file.write(sample_doc_content)
temp_file.close()

print("✓ Sample reference document created")
print(f"  Content preview: {sample_doc_content[:100]}...")

### Step 2.1: Generate Metadata with Gemini Flash

We use **gemini-2.0-flash-exp** for fast, cost-effective metadata generation.

In [ ]:
# Upload file to Gemini for analysis
print("Uploading file to Gemini for metadata generation...")
uploaded_file = client.files.upload(file_path=temp_file.name)

# Wait for processing
while uploaded_file.state == "PROCESSING":
    print("  Processing...")
    time.sleep(1)
    uploaded_file = client.files.get(name=uploaded_file.name)

print(f"✓ File processed: {uploaded_file.state}")

# Generate metadata using Gemini Flash
case_context = "This is a contract verification case for software development services between two companies."

metadata_prompt = f"""Analyze this document in the context of: {case_context}

Provide a JSON response with the following fields:
- summary: A 2-3 sentence summary of the document
- contextualization: How this document relates to the case context
- document_type: The type of document (e.g., contract, invoice, receipt, report)
- keywords: A list of 5-10 key terms or concepts from the document

Return only valid JSON, no markdown formatting."""

print("\nGenerating metadata with Gemini Flash...")
response = client.models.generate_content(
    model="gemini-2.0-flash-exp",
    contents=[uploaded_file, metadata_prompt]
)

# Parse metadata
response_text = response.text.strip()
if response_text.startswith("```"):
    response_text = response_text.split("```")[1]
    if response_text.startswith("json"):
        response_text = response_text[4:]
    response_text = response_text.strip()

metadata = json.loads(response_text)

print("\n✓ Metadata generated:")
print(f"  Document Type: {metadata['document_type']}")
print(f"  Summary: {metadata['summary']}")
print(f"  Keywords: {', '.join(metadata['keywords'][:5])}")

# Clean up the temporary analysis file
client.files.delete(name=uploaded_file.name)

### Step 2.2: Upload to File Search Store with Metadata

In [ ]:
# Upload file to Gemini Files API
print("Uploading file to Gemini Files API...")
file_for_store = client.files.upload(file_path=temp_file.name)

# Wait for processing
while file_for_store.state == "PROCESSING":
    time.sleep(1)
    file_for_store = client.files.get(name=file_for_store.name)

print(f"✓ File uploaded: {file_for_store.name}")

# Create custom metadata for File Search
custom_metadata = [
    types.CustomMetadata(
        key="summary",
        string_value=metadata['summary'][:500]  # Limit to 500 chars
    ),
    types.CustomMetadata(
        key="document_type",
        string_value=metadata['document_type']
    ),
    types.CustomMetadata(
        key="keywords",
        string_list_value=types.StringList(values=metadata['keywords'][:10])
    )
]

# Add file to File Search store
print("\nAdding file to File Search store...")
operation = client.file_search_stores.upload_to_file_search_store(
    file_search_store_name=CORPUS_STORE_NAME,
    file=file_for_store,
    config={
        'custom_metadata': custom_metadata,
        'display_name': 'Service Agreement - Company A & B'
    }
)

# Wait for indexing
print("Indexing document (this may take a few seconds)...")
max_wait = 60
elapsed = 0
while not operation.done and elapsed < max_wait:
    time.sleep(2)
    elapsed += 2
    operation = client.operations.get(name=operation.name)
    print(f"  Indexing... ({elapsed}s)")

if operation.done:
    print("\n✓ Document indexed successfully!")
    print(f"  Store: {CORPUS_STORE_NAME}")
    print(f"  File: {file_for_store.name}")
else:
    print("\n⚠️  Indexing still in progress (background processing)")

# Clean up temp file
Path(temp_file.name).unlink()

## Part 3: Querying the Corpus

Now that we have a corpus with indexed documents, let's query it using Gemini's File Search capability.

In [ ]:
# Define queries to test
queries = [
    "What is the payment amount mentioned in the contract?",
    "When does the project start and end?",
    "Who are the parties involved in this agreement?",
    "What are the payment terms?"
]

# Configure File Search tool
file_search_tool = types.Tool(
    file_search=types.FileSearch(
        file_search_store_names=[CORPUS_STORE_NAME]
    )
)

print("\n" + "=" * 80)
print("QUERYING THE CORPUS")
print("=" * 80)

for i, query in enumerate(queries, 1):
    print(f"\n📝 Query {i}: {query}")
    print("-" * 80)
    
    # Query with File Search
    response = client.models.generate_content(
        model="gemini-2.0-flash-exp",
        contents=query,
        config=types.GenerateContentConfig(
            tools=[file_search_tool]
        )
    )
    
    print(f"\n💡 Answer: {response.text}\n")
    
    # Show grounding sources if available
    if hasattr(response, 'candidates') and response.candidates:
        candidate = response.candidates[0]
        if hasattr(candidate, 'grounding_metadata') and candidate.grounding_metadata:
            if hasattr(candidate.grounding_metadata, 'grounding_chunks'):
                chunks = candidate.grounding_metadata.grounding_chunks
                if chunks:
                    print(f"📚 Sources ({len(chunks)} citations):")
                    for j, chunk in enumerate(chunks[:3], 1):  # Show first 3
                        if hasattr(chunk, 'document') and chunk.document:
                            title = getattr(chunk.document, 'title', 'Document')
                            print(f"  {j}. {title}")
                        if hasattr(chunk, 'content') and hasattr(chunk.content, 'text'):
                            excerpt = chunk.content.text[:150]
                            print(f"     \"{excerpt}...\"")

## Part 4: Verification Layer

The **verification layer** checks whether specific claims/chunks from a target document are supported by the corpus.

This is the core functionality of the Document Verification Assistant.

### Step 4.1: Define Chunks to Verify

These are statements from a document we want to verify against our corpus.

In [ ]:
# Define chunks to verify
chunks_to_verify = [
    {
        "page": 1,
        "item": "1",
        "text": "The contract was signed on January 15, 2024."
    },
    {
        "page": 1,
        "item": "2",
        "text": "Client agrees to pay $150,000 for the services."
    },
    {
        "page": 1,
        "item": "3",
        "text": "The project timeline is from February 1 to June 30, 2024."
    },
    {
        "page": 1,
        "item": "4",
        "text": "The parties agree to a 90-day warranty period."  # FALSE - not in document
    },
    {
        "page": 1,
        "item": "5",
        "text": "All work product becomes the property of the Client upon final payment."
    }
]

print(f"📋 Defined {len(chunks_to_verify)} chunks for verification")
for chunk in chunks_to_verify:
    print(f"  • Page {chunk['page']}, Item {chunk['item']}: {chunk['text'][:60]}...")

### Step 4.2: Verify Each Chunk Against Corpus

We use **Gemini Flash** with File Search to:
1. Check if the content is supported by the corpus
2. Assign a confidence score (1-10)
3. Provide source citations
4. Explain the reasoning

In [ ]:
print("\n" + "=" * 80)
print("VERIFICATION LAYER - Verifying Chunks Against Corpus")
print("=" * 80)

verification_results = []

for chunk in chunks_to_verify:
    print(f"\n{'='*80}")
    print(f"📄 Page {chunk['page']}, Item {chunk['item']}")
    print(f"📝 Chunk: \"{chunk['text']}\"")
    print(f"{'='*80}")
    
    # Build verification prompt
    verification_prompt = f"""Context: {case_context}

Verify this content against the reference documents:

Page {chunk['page']}, Item {chunk['item']}:
"{chunk['text']}"

Please verify if this content appears in or is supported by the reference documents. Provide your response in JSON format with these fields:

1. verified: (boolean) true if the content is found/supported, false otherwise
2. confidence_score: (integer 1-10) how confident you are in this verification
3. verification_source: (string) citation with document name and location
4. verification_note: (string) brief explanation of your reasoning
5. citations: (array) list of specific passages that support this verification, each with "title" and "excerpt" fields

Return only valid JSON, no markdown formatting."""
    
    # Verify with Gemini Flash + File Search
    response = client.models.generate_content(
        model="gemini-2.0-flash-exp",
        contents=verification_prompt,
        config=types.GenerateContentConfig(
            temperature=0.1,  # Low temperature for consistent results
            response_mime_type="application/json",
            tools=[file_search_tool]
        )
    )
    
    # Parse verification result
    result = json.loads(response.text)
    
    # Extract grounding citations
    actual_citations = []
    if hasattr(response, 'candidates') and response.candidates:
        candidate = response.candidates[0]
        if hasattr(candidate, 'grounding_metadata') and candidate.grounding_metadata:
            if hasattr(candidate.grounding_metadata, 'grounding_chunks'):
                for grounding_chunk in candidate.grounding_metadata.grounding_chunks:
                    citation = {}
                    if hasattr(grounding_chunk, 'document') and grounding_chunk.document:
                        citation["title"] = getattr(grounding_chunk.document, 'title', 'Document')
                    if hasattr(grounding_chunk, 'content') and hasattr(grounding_chunk.content, 'text'):
                        citation["excerpt"] = grounding_chunk.content.text
                    if citation:
                        actual_citations.append(citation)
    
    # Add to results
    result['chunk'] = chunk
    result['actual_citations'] = actual_citations
    verification_results.append(result)
    
    # Display result
    verified_icon = "✅" if result['verified'] else "❌"
    print(f"\n{verified_icon} VERIFIED: {result['verified']}")
    print(f"📊 Confidence Score: {result['confidence_score']}/10")
    print(f"📚 Source: {result['verification_source']}")
    print(f"💭 Note: {result['verification_note']}")
    
    if actual_citations:
        print(f"\n🔗 Grounding Citations ({len(actual_citations)}):")
        for i, citation in enumerate(actual_citations[:2], 1):  # Show first 2
            print(f"  {i}. {citation.get('title', 'Unknown')}")
            if 'excerpt' in citation:
                excerpt = citation['excerpt'][:150]
                print(f"     \"{excerpt}...\"")
    
    # Small delay to avoid rate limits
    time.sleep(0.5)

print("\n" + "=" * 80)
print("VERIFICATION COMPLETE")
print("=" * 80)

### Step 4.3: Verification Summary

In [ ]:
# Calculate statistics
total_chunks = len(verification_results)
verified_count = sum(1 for r in verification_results if r['verified'])
unverified_count = total_chunks - verified_count
avg_score = sum(r['confidence_score'] for r in verification_results) / total_chunks

print("\n" + "=" * 80)
print("📊 VERIFICATION SUMMARY")
print("=" * 80)
print(f"\nTotal Chunks: {total_chunks}")
print(f"✅ Verified: {verified_count} ({verified_count/total_chunks*100:.1f}%)")
print(f"❌ Unverified: {unverified_count} ({unverified_count/total_chunks*100:.1f}%)")
print(f"📊 Average Confidence: {avg_score:.1f}/10")

print("\n" + "-" * 80)
print("BREAKDOWN BY CHUNK:")
print("-" * 80)
for result in verification_results:
    chunk = result['chunk']
    icon = "✅" if result['verified'] else "❌"
    print(f"{icon} Page {chunk['page']}, Item {chunk['item']} - Score: {result['confidence_score']}/10")
    print(f"   \"{chunk['text'][:70]}...\"")
    print()

## Part 5: Export Verification Results

In [ ]:
# Export to JSON
output_data = {
    "corpus_store": CORPUS_STORE_NAME,
    "case_context": case_context,
    "total_chunks": total_chunks,
    "verified_chunks": verified_count,
    "average_confidence": avg_score,
    "results": verification_results
}

output_file = "verification_results.json"
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(output_data, f, indent=2, ensure_ascii=False)

print(f"✓ Verification results exported to: {output_file}")

## Cleanup: Delete File Search Store

**Important:** File Search stores are free, but you may want to clean up test stores.

In [ ]:
# Uncomment to delete the store
# client.file_search_stores.delete(name=CORPUS_STORE_NAME)
# print(f"✓ Deleted File Search store: {CORPUS_STORE_NAME}")

print("\n⚠️  Store not deleted. Uncomment code above to clean up.")
print(f"   Store ID: {CORPUS_STORE_NAME}")

## Summary

This notebook demonstrated:

1. ✅ **Creating a Corpus** - File Search Store for reference documents
2. ✅ **AI Metadata Generation** - Using Gemini Flash to analyze and tag documents
3. ✅ **Querying the Corpus** - Semantic search with File Search
4. ✅ **Verification Layer** - Verifying chunks against corpus with citations

### Key Models Used

- **gemini-2.0-flash-exp**: Fast, cost-effective model for metadata and verification
- **File Search**: Fully managed RAG with automatic chunking and embeddings

### Pricing

- **File Search Storage**: Free
- **File Search Querying**: Free
- **Initial Indexing**: $0.15 per 1M tokens
- **Flash Model**: $0.10 per 1M input tokens, $0.40 per 1M output tokens

### Next Steps

- Explore the full application in `backend/` and `frontend/`
- Run tests in `tests/` to see comprehensive examples
- Check `backend/app/gemini_service.py` for production implementation